In [1]:
import pickle

with open('data/sentiment_comp_qaie_pairs.pkl','rb') as f:
    pairs = pickle.loads(f.read())

In [2]:
print(len(pairs))
pairs = [p for p in pairs if p[1] not in ['无','']]
print(len(pairs))

pairs[0]

12325
12272


['智通财经APP讯，太平洋网络(00543)发布公告，将于2023年6月12日派发截至2022年12月31日止年度的末期股息每股0.1元人民币。\n---\n请从上文中抽取出所有公司/机构、对应的在本文中的情感倾向（积极、消极、中性）以及原因。\n并用这样的格式返回：\n{"ORG":..., "sentiment":..., "reason":...}',
 '{"ORG": "太平洋网络", "sentiment": "积极", "reason": "宣布派发股息每股0.1元人民币"}']

In [6]:
import json
with open('data/sentiment_comp_ie.json','w',encoding='utf8') as f:
    for p in pairs:
        line = {"q":p[0],"a":p[1]}
        f.write(json.dumps(line,ensure_ascii=False))
        f.write('\n')

In [3]:
# 打乱顺序重新排列
import json
import random
with open('data/sentiment_comp_ie_shuffled.json','w',encoding='utf8') as f:
    random_pairs = random.sample(pairs,k=len(pairs))
    for p in random_pairs:
        line = {"q":p[0],"a":p[1]}
        f.write(json.dumps(line,ensure_ascii=False))
        f.write('\n')

## baichuan-7B inference

In [1]:
# 下载百川大模型看看
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/baichuan-7B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("baichuan-inc/baichuan-7B", device_map="auto", trust_remote_code=True)

/root/anaconda3/envs/gby/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model

BaiChuanForCausalLM(
  (model): Model(
    (embed_tokens): Embedding(64000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (self_attn): Attention(
          (W_pack): Linear(in_features=4096, out_features=12288, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=64000, bias=False)
)

In [7]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "weights/sentiment_comp_ie_shuffled_baichuan-7B")

In [19]:
text = """ChatGPT的提出对谷嘎、万度的搜索业务产生巨大打击，传统搜索引擎的作用性降低了。
与此同时，OChat，Linguo等新兴语义搜索公司，迅速推出自己的类ChatGPT模型，并结合进自家搜索引擎，受到了很多用户的青睐。
腾势、艾里等公司表示会迅速跟进ChatGPT和AIGC的发展，并预计在年底前推出自己的大模型。
大型图片供应商视觉中国称ChatGPT对公司业务暂无影响，还在观望状态。
（本文图片来自视觉中国，上观新闻为您报道。）
更多报道：
- 亚牛逊公司关于AIGC的表态
- 巨硬公司昨日在A股上市
---
请从上文中抽取出所有公司，以及对应的在本文中的情感倾向（积极、消极、中性）以及原因。
请用这样的格式返回：
{"ORG":..., "sentiment":..., "reason":...}"""

In [18]:
from transformers import TextStreamer
streamer = TextStreamer(tokenizer,skip_prompt=True,skip_special_tokens=True)

inputs = tokenizer("我们说的艾里巴巴公司，指的是", return_tensors='pt')
inputs = inputs.to('cuda:0')
output = model.generate(**inputs, max_new_tokens=1024,repetition_penalty=1.1, streamer=streamer)
# 模型非常自信！（类似于模型自动纠错的能力）

阿里巴巴集团。2014年9月19日，马云在纽约联合国总部宣布：“阿里巴巴将投入一千亿人民币设立社会公益基金会”。


## ChatGLM inference

In [3]:
import torch
from transformers import AutoModel
device=torch.device(2)

model_path = "THUDM/chatglm-6b"
model_glm = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().to(device)
tokenizer_glm = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)


Loading checkpoint shards: 100%|██████████| 8/8 [00:07<00:00,  1.06it/s]


In [ ]:
# load LoRA
model_glm = PeftModel.from_pretrained(model_glm, "weights/sentiment_comp_ie").half()

In [19]:
from transformers import TextStreamer
streamer = TextStreamer(tokenizer_glm,skip_prompt=True,skip_special_tokens=True)

inputs = tokenizer_glm("我们说的艾里巴巴公司，指的是", return_tensors='pt')
inputs = inputs.to('cuda:2')
output = model_glm.generate(**inputs, max_new_tokens=1024,repetition_penalty=1.1, streamer=streamer)

我不确定您所指的“艾里巴巴公司”是指哪个公司。如果您能提供更多上下文或信息,我将尽力回答您的问题。


In [17]:
print(text)

融资资金将用于完善生成式AI新引擎的构建。

创业邦获悉，自然语言技术AI服务商竹间智能宣布已完成D2轮融资，由金浦投资、金库资本、江苏文投、隽赐资本等联合投资。至今，竹间智能已累计完成7轮融资，陆续引入科沃斯、云晖资本、中银国际等股东机构。值得一提的是，竹间智能已经正式推出运用类ChatGPT技术的成熟AIGC产品，并即将开启新一轮融资。

融资资金将用于完善生成式AI新引擎的构建，把以ChatGPT为代表的大语言模型和AIGC技术全面融入竹间产品体系，结合大小模型构建双引擎驱动产品迭代和技术升级，平衡大小模型的优缺点，全面焕新产品功能和服务模式，并正式将产品同步推向海外市场，成为服务全球企业和用户的跨国NLP能力厂商。

竹间智能由前微软（亚洲）互联网工程院副院长简仁贤于2015年创办，致力于以自然语言处理、情感计算、深度学习、知识工程、文本处理等人工智能技术为基础，将AI能力惠及千行百业。

---
请从上文中抽取出所有公司，以及对应的在本文中的情感倾向（积极、消极、中性）以及原因。
请用这样的格式返回：
{"ORG":..., "sentiment":..., "reason":...}


tokenizer自动把中文括号转成英文括号了...所以你直接在输出里面可能无法直接匹配到 prompt